In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# -----------------
# Parameters
# -----------------
nx = 201
L = 2.0
x = np.linspace(0, L, nx)
dx = x[1] - x[0]
CFL = 0.9
flow_time = 0.3
nu = 0.001  # viscosity; set 0 for inviscid

# Initial condition: shock-tube-like
u0 = np.ones_like(x)
u0[x > 0.5*L] = 2.0
u = u0.copy()

umax = np.max(np.abs(u0))
dt_adv = CFL * dx / umax
dt_diff = CFL * dx**2 / nu if nu > 0 else np.inf
dt = min(dt_adv, dt_diff)
nt = int(flow_time / dt) + 1

print(f"dx={dx:.4f}, dt={dt:.4e}, nt={nt}, nu={nu}")

# -----------------
# Control number of frames
# -----------------
desired_frames = 200
snapshot_interval = max(nt // desired_frames, 1)
actual_frames = nt // snapshot_interval
print(f"Desired frames={desired_frames}, snapshot interval={snapshot_interval}, actual frames={actual_frames}")

# -----------------
# Set up the figure
# -----------------
fig, ax = plt.subplots()
line, = ax.plot(x, u0, lw=2)
ax.set_xlim(0, L)
ax.set_ylim(0, 2.2)
ax.set_xlabel("x")
ax.set_ylabel("u")
title = ax.set_title("Time: 0.000")

# -----------------
# Time-stepping function
# -----------------
frame_times = np.arange(0, nt, snapshot_interval)  # list of time steps for frames

def update(frame_idx):
    global u
    frame = frame_times[frame_idx]
    for t in range(snapshot_interval):
        un = u.copy()
        dudx = np.zeros_like(u)
        dudx[1:-1] = (un[2:] - un[:-2]) / (2*dx)
        d2udx2 = np.zeros_like(u)
        d2udx2[1:-1] = (un[2:] - 2*un[1:-1] + un[:-2]) / dx**2
        u[1:-1] = un[1:-1] - un[1:-1] * dt * dudx[1:-1] + nu * dt * d2udx2[1:-1]
        u[0] = u0[0]
        u[-1] = u0[-1]
    line.set_ydata(u)
    time = frame * dt
    title.set_text(f"Time: {time:.3f}")
    return line, title

ani = FuncAnimation(fig, update, frames=len(frame_times), interval=30, blit=True)

# -----------------
# Save animation to HTML
# -----------------
html_anim = ani.to_html5_video()
HTML(html_anim)



dx=0.0100, dt=4.5000e-03, nt=67, nu=0.001
Desired frames=200, snapshot interval=1, actual frames=67


<IPython.core.display.Javascript object>